# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [56]:
# Import required libraries
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [57]:
# Initialize environment and API
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
# Initialize OpenAI client
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


In [58]:
# Available models
MODELS = {
    "GPT-4o Mini": "gpt-4o-mini",
    "GPT-4o": "gpt-4o",
    "GPT-3.5 Turbo": "gpt-3.5-turbo"
}

# Default model
DEFAULT_MODEL = "gpt-4o-mini"

# System prompts for different expertise
SYSTEM_PROMPTS = {
    "Airline Assistant": "You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so.",
    "Technical Expert": "You are a technical expert specializing in computer science, programming, and software development. Provide detailed, accurate technical information with code examples when appropriate.",
    "Language Tutor": "You are a language tutor helping students learn new languages. Explain grammar concepts clearly, provide examples, and correct mistakes in a supportive way.",
    "Custom": ""  # Will be filled by user input
}

# Default system prompt
DEFAULT_SYSTEM_PROMPT = "Airline Assistant"


In [59]:
# Tool functions
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    """Get the price of a ticket to the specified destination city."""
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# Function definition for OpenAI
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# List of tools
tools = [{"type": "function", "function": price_function}]


In [60]:
def handle_tool_call(message):
    """Handle tool calls from the OpenAI API with robust error handling."""
    try:
        # Check if tool_calls exists and is not empty
        if not hasattr(message, 'tool_calls') or not message.tool_calls:
            print("Warning: No tool calls found in message")
            return {"role": "assistant", "content": "I couldn't process that request."}, None
        
        tool_call = message.tool_calls[0]
        
        # Check if function and arguments exist
        if not hasattr(tool_call, 'function') or not hasattr(tool_call.function, 'arguments'):
            print("Warning: Tool call missing function or arguments")
            return {"role": "assistant", "content": "I couldn't process that request."}, None
        
        # Safely parse arguments with error handling
        try:
            # Check if arguments is empty
            if not tool_call.function.arguments or tool_call.function.arguments.strip() == "":
                print("Warning: Empty arguments in tool call")
                arguments = {}
            else:
                arguments = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError as e:
            print(f"Warning: Failed to parse tool call arguments: {e}")
            print(f"Arguments received: '{tool_call.function.arguments}'")
            arguments = {}
        
        # Get city with fallback
        city = arguments.get('destination_city', "unknown")
        price = get_ticket_price(city)
        
        # Create response with proper tool_call_id
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "price": price}),
            "tool_call_id": tool_call.id if hasattr(tool_call, 'id') else "unknown_id"
        }
        
        return response, city
    except Exception as e:
        # Catch-all for any other unexpected errors
        print(f"Error in handle_tool_call: {str(e)}")
        return {"role": "assistant", "content": "I encountered an error while processing your request."}, None


In [61]:
def chat_with_streaming(message, history, system_prompt_key, custom_system_prompt, model_name):
    """Chat function with streaming support."""
    # Determine which system prompt to use
    if system_prompt_key == "Custom":
        system_message = custom_system_prompt
    else:
        system_message = SYSTEM_PROMPTS[system_prompt_key]
    
    # Convert history to the format expected by OpenAI
    messages = [{"role": "system", "content": system_message}]
    
    # Process history - ensure it's in the right format for OpenAI
    # Handle both tuple format and dictionary format for backward compatibility
    if history:
        for msg in history:
            if isinstance(msg, dict):
                # If it's already a dict with role and content, use it directly
                if "role" in msg and "content" in msg:
                    messages.append(msg)
            elif isinstance(msg, (list, tuple)) and len(msg) == 2:
                # If it's a tuple/list of (user_msg, assistant_msg), convert to dicts
                user_msg, assistant_msg = msg
                messages.append({"role": "user", "content": user_msg})
                messages.append({"role": "assistant", "content": assistant_msg})
    
    # Add the current message
    messages.append({"role": "user", "content": message})
    
    # Get the model from the selected name
    model = MODELS[model_name]
    
    # Create a streaming response
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        stream=True
    )
    
    # Variables to track the response
    collected_messages = []
    finish_reason = None
    message_obj = None
    
    # Process the streaming response
    for chunk in response:
        if chunk.choices[0].delta.content:
            collected_messages.append(chunk.choices[0].delta.content)
            partial_message = "".join(collected_messages)
            yield partial_message
        
        # Check if we have a tool call
        if chunk.choices[0].finish_reason:
            finish_reason = chunk.choices[0].finish_reason
        
        # Store the message object if it has tool calls
        if hasattr(chunk.choices[0].delta, 'tool_calls') and chunk.choices[0].delta.tool_calls:
            if message_obj is None:
                message_obj = chunk.choices[0].delta
            else:
                # Append tool call information
                if not hasattr(message_obj, 'tool_calls'):
                    message_obj.tool_calls = []
                message_obj.tool_calls.extend(chunk.choices[0].delta.tool_calls)
    
    # If we need to handle a tool call
    if finish_reason == "tool_calls" and message_obj and hasattr(message_obj, 'tool_calls'):
        # Handle the tool call
        tool_response, city = handle_tool_call(message_obj)
        
        # Add the tool call and response to messages
        messages.append({"role": "assistant", "content": None, "tool_calls": message_obj.tool_calls})
        messages.append(tool_response)
        
        # Get a new response from the model
        second_response = openai.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
        
        # Reset collected messages
        collected_messages = []
        
        # Process the second streaming response
        for chunk in second_response:
            if chunk.choices[0].delta.content:
                collected_messages.append(chunk.choices[0].delta.content)
                partial_message = "".join(collected_messages)
                yield partial_message
    
    # Return the final message
    final_message = "".join(collected_messages)
    return final_message


In [62]:
def create_ui():
    """Create the Gradio UI for the application."""
    with gr.Blocks(title="Technical Question/Answerer") as demo:
        gr.Markdown("# Technical Question/Answerer Prototype")
        gr.Markdown("Ask questions and get answers with streaming responses, model switching, and tool integration.")
        
        with gr.Row():
            with gr.Column(scale=1):
                # System prompt selection
                system_prompt_dropdown = gr.Dropdown(
                    choices=list(SYSTEM_PROMPTS.keys()),
                    value=DEFAULT_SYSTEM_PROMPT,
                    label="System Prompt (Expertise)"
                )
                
                # Custom system prompt input
                custom_system_prompt = gr.Textbox(
                    lines=3,
                    placeholder="Enter your custom system prompt here...",
                    label="Custom System Prompt",
                    visible=False
                )
                
                # Model selection
                model_dropdown = gr.Dropdown(
                    choices=list(MODELS.keys()),
                    value="GPT-4o Mini",
                    label="Model"
                )
            
            with gr.Column(scale=2):
                # Chat interface - explicitly set type to 'messages' to avoid deprecation warning
                chatbot = gr.Chatbot(height=500, type="messages")
                msg = gr.Textbox(placeholder="Type your message here...", label="Message")
        
        # Event handlers
        def update_custom_prompt_visibility(prompt_key):
            return {"visible": prompt_key == "Custom"}
        
        system_prompt_dropdown.change(
            fn=update_custom_prompt_visibility,
            inputs=system_prompt_dropdown,
            outputs=custom_system_prompt
        )
        
        # Handle text input - ensure we're updating the chatbot correctly with message dictionaries
        def process_chat(message, history, system_prompt_key, custom_system_prompt, model_name):
            # Initialize history if None
            history = history or []
            
            # Get the streaming response
            response_generator = chat_with_streaming(message, history, system_prompt_key, custom_system_prompt, model_name)
            
            # Add user message as a dictionary with role and content
            history.append({"role": "user", "content": message})
            # Add initial empty assistant message
            history.append({"role": "assistant", "content": ""})
            
            # Update the assistant's message as the response streams in
            for partial_response in response_generator:
                history[-1]["content"] = partial_response
                yield history
        
        msg.submit(
            fn=process_chat,
            inputs=[msg, chatbot, system_prompt_dropdown, custom_system_prompt, model_dropdown],
            outputs=chatbot,
            queue=False
        ).then(
            lambda: "",
            None,
            msg
        )
        
    return demo


In [63]:
# Create and launch the UI
# IMPORTANT: Do NOT use share=True to avoid antivirus conflicts
demo = create_ui()
demo.launch()  # Removed share=True to avoid antivirus conflicts


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\simo_\anaconda3\envs\llms\Lib\site-packages\gradio\routes.py", line 1191, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\simo_\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\simo_\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2146, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\simo_\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1676, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\simo_\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py", line 729, in async_iteration
    return await anext(iterator)
         

In [64]:
# Test the ticket price tool
print("Testing ticket price tool:")
cities = ["London", "Paris", "Tokyo", "Berlin", "Unknown City"]
for city in cities:
    price = get_ticket_price(city)
    print(f"Price for {city}: {price}")


Testing ticket price tool:
Tool get_ticket_price called for London
Price for London: $799
Tool get_ticket_price called for Paris
Price for Paris: $899
Tool get_ticket_price called for Tokyo
Price for Tokyo: $1400
Tool get_ticket_price called for Berlin
Price for Berlin: $499
Tool get_ticket_price called for Unknown City
Price for Unknown City: Unknown
Tool get_ticket_price called for unknown
